# 📚 Kapitola 4: Python - instalace a virtuální prostředí

<div style="background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h2 style="color: white; margin: 0;">Blok 1 | BEGINNER</h2>
    <p style="color: white; margin: 10px 0;">📖 Výuková kapitola</p>
</div>

## 🎯 Co se naučíte

V této kapitole se zaměříme na následující témata:

- **Python 3.11+ instalace**
- **Windows: py launcher, python.exe**
- **MacOS: pyenv, python3**
- **Ubuntu: python3, python3-pip**
- **Virtuální prostředí: venv, virtualenv, conda**
- **pip a pip-tools**
- **requirements.txt a pyproject.toml**

## ⚠️ Předpoklady
Tato kapitola navazuje na kapitoly: 1, 2

---


In [ ]:
# ⚙️ Inicializace prostředí
# Tento kód nastavuje prostředí pro kapitolu 4

import sys
import os
from datetime import datetime

# Informace o prostředí
print("=" * 60)
print(f"📚 KAPITOLA 4: Python - instalace a virtuální prostředí")
print("=" * 60)
print(f"🐍 Python verze: {sys.version}")
print(f"📅 Datum spuštění: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💻 OS: {os.name}")
print(f"📁 Pracovní adresář: {os.getcwd()}")
print("=" * 60)

# Instalace potřebných knihoven (odkomentujte podle potřeby)
# !pip install requests pandas numpy matplotlib
# !pip install beautifulsoup4 sqlalchemy fastapi

# Import základních knihoven
import json
import csv
import time
import random
from pathlib import Path
from typing import List, Dict, Optional, Any

print("✅ Prostředí připraveno!")

## 📖 Teoretická část

<div style="background: #f0f4ff; padding: 20px; border-left: 5px solid #4a69bd; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🎓 Základní teorie a koncepty</h3>
</div>

# Python - instalace a virtuální prostředí

## 1. Úvod a motivace

V tomto tutoriálu se naučíš, jak nainstalovat Python na různé operační systémy a jak pracovat s virtuálním prostředím – důležitým nástrojem pro vývoj Python aplikací. Pokud jsi začátečník, pak je tato kapitola klíčová, protože základní instalace a správa prostředí jsou prvním krokem ke spuštění tvého prvního Python programu.

Python je velmi populární jazyk pro různorodé úlohy – od automatizací, přes webové aplikace až po strojové učení. Aby ses mohl s Pythonem začít, musíš si ho správně nainstalovat a zajistit, aby tvé programy neměly problémy s závislostmi a kompatibilitou.

V praxi se vývojáři v Pythonu neustále setkávají s potřebou používat různé verze knihoven, které mohou být nekompatibilní mezi sebou. Proto je důležité udržovat izolovaná prostředí pro každý projekt – tedy virtuální prostředí. Naučíš se, jak si nainstalovat Python a jak ho používat s využitím virtuálních prostředí, aby tvé aplikace byly spolehlivé a bezpečné.

Po této kapitole budeš schopen:
- Instalovat Python na Windows, macOS i Linux
- Vytvářet a spravovat virtuální prostředí
- Používat nástroje pro správu závislostí jako `pip` a `requirements.txt`
- Pracovat s moderními konfiguračními soubory jako `pyproject.toml`

---

## 2. Hlavní koncepty

### Python 3.11+ instalace

Python je programovací jazyk, který se musí nejprve nainstalovat do systému, aby mohl běžet na tvém počítači. V tomto tutoriálu se zaměříme na instalaci Pythonu 3.11 a vyšších verzí, protože jsou stabilní a obsahují nové funkce.

#### Windows

Na Windows systému je nejlepší způsob instalace Pythonu přes oficiální web [python.org](https://www.python.org/downloads/). Můžeš si stáhnout instalační program `python-3.x.x-amd64.exe`.

Při instalaci se ujisti, že je zaškrtnutý **"Add Python to PATH"** – tato volba ti umožní spouštět Python přímo z příkazové řádky (např. `python` nebo `python3`). Můžeš si také vybrat možnost instalace **Python Launcher for Windows (py)**, který ti umožní spouštět různé verze Pythonu podle potřeby.

Příklad spuštění z příkazové řádky:
```bash
C:\> python --version
Python 3.11.0
```

#### macOS

Na macOS je možné nainstalovat Python dvěma hlavními způsoby:

1. **Přes oficiální instalátor** – podobně jako na Windows.
2. **Pomocí nástroje `pyenv`**, který umožňuje spravovat více verzí Pythonu současně.

Nainstaluješ-li Python přes oficiální instalátor, můžeš získat přístup ke komandě `python3` (nebo `python`, pokud je to nastaveno v PATH).

Příklad:
```bash
$ python3 --version
Python 3.11.0
```

Pokud chceš spravovat více verzí Pythonu, můžeš použít `pyenv`:
```bash
# Instalace pyenv (na macOS)
brew install pyenv

# Nastavení verze Pythonu
pyenv install 3.11.0
pyenv global 3.11.0
```

#### Ubuntu (Linux)

Na Linuxu, konkrétně na Ubuntu, se Python instaluje pomocí správce balíčků `apt`. Můžeš nainstalovat Python i pip:
```bash
sudo apt update
sudo apt install python3 python3-pip
```

Zkontroluj verzi:
```bash
$ python3 --version
Python 3.11.0
```

Použití `pip`:
```bash
$ pip3 --version
pip 22.0.2 from /usr/lib/python3/dist-packages/pip (python 3.11)
```

---

### Virtuální prostředí: venv, virtualenv, conda

Virtuální prostředí je izolované prostředí pro Python projekt, které obsahuje vlastní instalaci Pythonu a balíčků. Tím zabíháš z rizik spojených s kompatibilitou mezi různými knihovnami.

#### `venv` – přirozený nástroj Pythonu (od verze 3.3+)

`venv` je modul, který je součástí standardního Pythonu. Vytvoření virtuálního prostředí je jednoduché:

```bash
python3 -m venv myproject_env
```

Aktivace:
- **Windows**:
  ```cmd
  myproject_env\Scripts\activate
  ```
- **macOS/Linux**:
  ```bash
  source myproject_env/bin/activate
  ```

Po aktivaci se v příkazové řádce objeví název prostředí, např. `(myproject_env) $`.

#### `virtualenv` – alternativa k `venv`

`virtualenv` je externí nástroj, který můžeš nainstalovat pomocí pip:
```bash
pip install virtualenv
```

Vytvoření prostředí:
```bash
virtualenv myproject_env
```

Aktivace:
```bash
source myproject_env/bin/activate  # Linux/macOS
myproject_env\Scripts\activate     # Windows
```

#### `conda` – správa prostředí pomocí Anaconda

`conda` je nástroj pro správu balíčků a virtuálních prostředí, často používaný v oblasti datové analýzy. Instaluješ ho přes Anaconda nebo Miniconda.

Vytvoření prostředí:
```bash
conda create -n myproject python=3.11
```

Aktivace:
```bash
conda activate myproject
```

---

### pip a pip-tools

`pip` je nástroj pro instalaci Python balíčků z [PyPI](https://pypi.org/) (Python Package Index).

#### Základní příkazy:

- Instalace balíčku:
  ```bash
  pip install requests
  ```

- Instalace konkrétní verze:
  ```bash
  pip install django==4.1.0
  ```

- Uložení seznamu závislostí do souboru `requirements.txt`:
  ```bash
  pip freeze > requirements.txt
  ```

#### `pip-tools` – pokročilá správa závislostí

`pip-tools` umožňuje spravovat závislosti vývojově čistěji. Například máš soubor `requirements.in`, kde definuješ závislosti, a pak spustíš:

```bash
pip-compile requirements.in
```

Vytvoří se automaticky soubor `requirements.txt` s přesnými verzemi.

---

### requirements.txt a pyproject.toml

#### `requirements.txt`

Tento soubor obsahuje seznam všech balíčků potřebných pro projekt. Můžeš ho použít ke snadnému nasazení nebo obnovení prostředí:
```bash
pip install -r requirements.txt
```

Příklad obsahu:
```txt
Django==4.1.0
requests==2.28.0
numpy==1.24.0
```

#### `pyproject.toml`

Modernější způsob správy projektu a jeho závislostí – používá se v moderních Python projektech.

Příklad:
```toml
[project]
name = "myproject"
version = "0.1.0"
dependencies = [
    "Django>=4.1.0",
    "requests>=2.28.0"
]
```

---

## 3. Důležité detaily

### Časté chyby a tipy na jejich řešení

- **Chyba: `python command not found`** – znamená to, že Python není v PATH. Ujisti se, že jsi při instalaci zaškrtnul „Add Python to PATH“.
- **Nesprávné prostředí aktivované** – můžeš se dostat do situace, kdy používáš nesprávnou verzi Pythonu. Vždy zkontroluj, zda je aktivní správné virtuální prostředí.
- **Použití `pip` místo `pip3`** – na systémech, kde je nainstalován i Python 2, je důležité používat `pip3` pro Python 3.

### Best practices

1. Vždy používej virtuální prostředí pro každý projekt.
2. Spravuj závislosti pomocí `requirements.txt` nebo `pyproject.toml`.
3. Používej `pip-tools` pro správu verzí a minimalizaci konfliktů.
4. Nenainstalovávej balíčky globálně – vždy do virtuálního prostředí.

### Performance tipy

- Využívej `venv` místo `virtualenv`, protože je součástí standardní Python knihovny.
- Nezapomínej aktualizovat `pip`:
  ```bash
  pip install --upgrade pip
  ```
- Používej `pip-tools` pro lepší kontrolu závislostí a minimalizaci chyb.

---

## 4. Propojení s předchozími kapitolami

Tato kapitola je logickým pokračováním výuky Pythonu, kterou jsi začal v úvodní kapitole. Zde se naučíš základní nástroje, které ti umožní spustit a spravovat Python skripty – důležité pro další kapitoly o funkcích, modulůch, práci s daty a webovými aplikacemi.

Použití virtuálních prostředí rozšiřuje základní znalosti o tom, jak spouštět Python, a zároveň připravuje na pokročilé techniky práce s Python projekty. Všechny tyto nástroje tvoří základ pro efektivní a bezpečné vývojové prostředí.

--- 

Tato kapitola ti dává základ, který potřebuješ pro tvorbu Python aplikací – od instalace přes správu prostředí až po správu balíčků. Pokračuj dál a vydáš se na cestu k vlastnímu Python projektu!


## 💻 Praktické příklady

<div style="background: #e8f5e9; padding: 20px; border-left: 5px solid #4caf50; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">👨‍💻 Hands-on příklady ke spuštění</h3>
    <p>Následující příklady si můžete hned vyzkoušet. Každý příklad je samostatně spustitelný.</p>
</div>


### Příklad 1

### Kompletní kód:

In [ ]:
# Tento skript zkontroluje instalaci Pythonu na systému Windows
import sys

print("Verze Pythonu nainstalovaného v systému:")
print(sys.version)

# Případně můžeme použít py launcher, např. z příkazové řádky:
# py --version
# python --version

# Výsledek bude něco jako:
# Python 3.11.4 (tags/v3.11.4:d2340ef, Jun  7 2023, 16:49:15) [MSC v.1934 64 bit (AMD64)]

### Příklad 2

### Očekávaný výstup:
```
Verze Pythonu nainstalovaného v systému:
Python 3.11.4 (tags/v3.11.4:d2340ef, Jun  7 2023, 16:49:15) [MSC v.1934 64 bit (AMD64)]
```

### Příklad 3

### Vysvětlení:
Skript používá `sys.version`, který vrací informaci o aktuálně spuštěném Pythonu. Tento způsob je obecný a funguje na všech platformách.

---

# 2. Název příkladu: Vytvoření virtuálního prostředí pomocí `venv`  
## Co demonstruje:
Vytvoření jednoduchého virtuálního prostředí pomocí vestavěného modulu `venv`.

### Příklad 4

### Kompletní kód:

In [ ]:
# Tento skript ukazuje, jak vytvořit virtuální prostředí s použitím 'venv'
import subprocess
import sys

# Vytvoření virtuálního prostředí
subprocess.run([sys.executable, "-m", "venv", "my_venv"])

print("Virtuální prostředí 'my_venv' bylo vytvořeno.")

### Příklad 5

### Očekávaný výstup:
```
Virtuální prostředí 'my_venv' bylo vytvořeno.
```

### Příklad 6

### Vysvětlení:
Používáme `subprocess.run()` k spuštění příkazu, který vytvoří virtuální prostředí s názvem `my_venv`. Tento příkaz používá vestavěný modul `venv`, který je dostupný ve všech verzích Pythonu 3.3+.

---

# 3. Název příkladu: Instalace balíčku do virtuálního prostředí  
## Co demonstruje:
Použití `pip` k instalaci balíčku ve virtuálním prostředí.

### Příklad 7

### Kompletní kód:

In [ ]:
# Tento skript demonstuuje instalaci balíčku do virtuálního prostředí
import subprocess
import sys

# Příprava: Aktivujeme virtuální prostředí a nainstalujeme requests
print("Instaluji balíček 'requests'...")
subprocess.run([sys.executable, "-m", "pip", "install", "requests"])

print("Balíček requests byl úspěšně nainstalován.")

### Příklad 8

### Očekávaný výstup:
```
Instaluji balíček 'requests'...
Collecting requests
  Downloading requests-2.31.0-py3-none-any.whl (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 1.2 MB/s eta 0:00:00
Installing collected packages: requests
Successfully installed requests-2.31.0
Balíček requests byl úspěšně nainstalován.
```

### Příklad 9

### Vysvětlení:
Skript spouští příkaz `pip install requests`, který nainstaluje balíček `requests`. Tento balíček je často používán pro práci s HTTP požadavky.

---

# 4. Název příkladu: Vytvoření a použití requirements.txt  
## Co demonstruje:
Vytvoření souboru `requirements.txt` a instalace balíčků z něj.

### Příklad 10

### Kompletní kód:

In [ ]:
# Tento skript vytvoří soubor requirements.txt a nainstaluje balíčky z něj

import subprocess
import sys

# Vytvoření souboru requirements.txt
requirements_content = """
requests==2.31.0
numpy==1.24.3
pandas==2.0.3
"""

with open("requirements.txt", "w") as f:
    f.write(requirements_content)

print("Soubor requirements.txt byl vytvořen.")

# Instalace balíčků z requirements.txt
subprocess.run([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])

print("Balíčky z requirements.txt byly úspěšně nainstalovány.")

### Příklad 11

### Očekávaný výstup:
```
Soubor requirements.txt byl vytvořen.
Collecting requests==2.31.0
  Downloading requests-2.31.0-py3-none-any.whl (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 1.2 MB/s eta 0:00:00
Collecting numpy==1.24.3
  Downloading numpy-1.24.3-cp311-cp311-win_amd64.whl (15.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 2.0 MB/s eta 0:00:00
Collecting pandas==2.0.3
  Downloading pandas-2.0.3-cp311-cp311-win_amd64.whl (11.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 2.0 MB/s eta 0:00:00
Installing collected packages: requests, numpy, pandas
Successfully installed numpy-1.24.3 pandas-2.0.3 requests-2.31.0
Balíčky z requirements.txt byly úspěšně nainstalovány.
```

### Příklad 12

### Vysvětlení:
Skript vytvoří soubor `requirements.txt`, který obsahuje seznam balíčků a jejich verzí. Pak tento soubor použije při instalaci balíčků pomocí `pip install -r requirements.txt`.

---

# 5. Název příkladu: Vytvoření projektu s pyproject.toml  
## Co demonstruje:
Vytvoření konfiguračního souboru `pyproject.toml` a použití `pip-tools` pro správu závislostí.

### Příklad 13

### Kompletní kód:

In [ ]:
# Tento skript vytvoří soubor pyproject.toml a nainstaluje pip-tools

import subprocess
import sys

# Vytvoření souboru pyproject.toml
pyproject_content = """
[build-system]
requires = ["setuptools>=45", "wheel"]
build-backend = "setuptools.build_meta"

[project]
name = "my_project"
version = "0.1.0"
dependencies = [
    "requests==2.31.0",
    "pandas==2.0.3",
]

[project.optional-dependencies]
dev = [
    "pip-tools>=7.4.0",
]
"""

with open("pyproject.toml", "w") as f:
    f.write(pyproject_content)

print("Soubor pyproject.toml byl vytvořen.")

# Instalace pip-tools
subprocess.run([sys.executable, "-m", "pip", "install", "pip-tools"])

print("Balíček pip-tools byl nainstalován.")

### Příklad 14

### Očekávaný výstup:
```
Soubor pyproject.toml byl vytvořen.
Collecting pip-tools>=7.4.0
  Downloading pip_tools-7.4.1-py3-none-any.whl (24 kB)
Installing collected packages: pip-tools
Successfully installed pip-tools-7.4.1
Balíček pip-tools byl nainstalován.
```

### Příklad 15

### Vysvětlení:
Skript vytvoří soubor `pyproject.toml`, který je standardním způsobem konfigurace Python projektu. Používá se spolu s `pip-tools` pro správu závislostí, což umožňuje snadné aktualizace a sledování verzí balíčků.

---

## 🎯 Cvičení a úkoly

<div style="background: #fff3e0; padding: 20px; border-left: 5px solid #ff9800; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">✍️ Praktická cvičení k procvičení</h3>
    <p>Vyřešte následující úkoly. Začněte od jednoduššších a postupujte k složitějším.</p>
</div>

# 5 praktických cvičení: Python - instalace a virtuální prostředí

---

## Cvičení 1: Instalace Pythonu a ověření verze

### **Název úkolu**
Instalace Pythonu a kontrola verze

### **Detailní zadání**
Student má nainstalovat Python na svůj počítač (pokud ještě není nainstalovaný) a ověřit, že je instalace správná. Poté vytvoří jednoduchý skript, který vypíše verzi Pythonu.

### **Vstupní data/požadavky**
- Uživatel nemusí zadávat žádné vstupní data
- Musí být nainstalovaný Python (verze 3.7 nebo vyšší)

### **Očekávaný výstup**
Výpis verze Pythonu, např.:
```
Python 3.11.4
```

### **Hints/Nápověda**
1. Stáhni a nainstaluj Python z [python.org](https://www.python.org/)
2. Otevři terminál/příkazovou řádku (Command Prompt nebo Terminal)
3. Zadej příkaz `python --version` nebo `python3 --version`
4. Vytvoř soubor `version_check.py` a vlož kód pro získání verze

### **Kostra řešení**
```python
import sys
print("Python verze:", sys.version)
```

### **Bonusové rozšíření**
Vytvoř skript, který kontroluje i dostupnost nástroje `pip` a vypíše jeho verzi.

---

## Cvičení 2: Vytvoření virtuálního prostředí

### **Název úkolu**
Vytvoření a aktivace virtuálního prostředí

### **Detailní zadání**
Student vytvoří virtuální prostředí pomocí modulu `venv`, aktivuje ho a ověří, že je aktivní.

### **Vstupní data/požadavky**
- Nainstalovaný Python (verze 3.7+)
- Přístup k příkazové řádce

### **Očekávaný výstup**
- Vytvořen složka `myenv` (nebo jiný název)
- Aktivované virtuální prostředí (např. v příkazové řádce se objeví `(myenv)`)

### **Hints/Nápověda**
1. V příkazové řádce zadej `python -m venv myenv`
2. Aktivuj prostředí:  
   - Windows: `myenv\Scripts\activate`  
   - Mac/Linux: `source myenv/bin/activate`
3. Ověř aktivaci příkazem `where python` (Windows) nebo `which python` (Mac/Linux)
4. Všimni si, že se změnila cesta k Pythonu

### **Kostra řešení**
```bash
python -m venv myenv
# Aktivuj virtuální prostředí podle OS
# Poté zkontroluj aktuální Python cestu
```

### **Bonusové rozšíření**
Vytvoř skript, který automaticky zkontroluje, zda je virtuální prostředí aktivní a vypíše jeho cestu.

---

## Cvičení 3: Instalace balíčku v virtuálním prostředí

### **Název úkolu**
Instalace a použití balíčku `requests`

### **Detailní zadání**
Student aktivuje virtuální prostředí, nainstaluje balíček `requests` pomocí `pip`, a vytvoří jednoduchý skript, který provede HTTP požadavek na ukázkovou URL.

### **Vstupní data/požadavky**
- Aktivní virtuální prostředí
- Přístup k internetu

### **Očekávaný výstup**
Skript vypíše HTTP status kód a tělo odpovědi (např. text nebo JSON)

### **Hints/Nápověda**
1. Zkontroluj, že je virtuální prostředí aktivní (`pip` by měl být dostupný)
2. Nainstaluj balíček: `pip install requests`
3. Použij `requests.get()` na URL jako např. `https://httpbin.org/get`
4. Vypiš status a text odpovědi

### **Kostra řešení**
```python
import requests
response = requests.get("https://httpbin.org/get")
print("Status kód:", response.status_code)
print("Odpověď:", response.text)
```

### **Bonusové rozšíření**
Použij `requests` k vytvoření jednoduchého API klienta, který si stáhne JSON data a vypíše konkrétní položky.

---

## Cvičení 4: Vytvoření a aktivace virtuálního prostředí pomocí `virtualenv`

### **Název úkolu**
Vytvoření virtuálního prostředí s `virtualenv`

### **Detailní zadání**
Student nainstaluje balíček `virtualenv` a vytvoří virtuální prostředí pomocí příkazu `virtualenv`. Poté ho aktivuje a ověří.

### **Vstupní data/požadavky**
- Nainstalovaný Python
- Přístup k příkazové řádce

### **Očekávaný výstup**
- Složka s virtuálním prostředím (např. `venv_env`)
- Aktivované prostředí ve výstupu příkazu `python --version`

### **Hints/Nápověda**
1. Nainstaluj `virtualenv` pomocí `pip install virtualenv`
2. Vytvoř virtuální prostředí: `virtualenv venv_env`
3. Aktivuj prostředí (Windows: `venv_env\Scripts\activate`, Mac/Linux: `source venv_env/bin/activate`)
4. Zkontroluj, že se používá správný Python

### **Kostra řešení**
```bash
pip install virtualenv
virtualenv venv_env
# Aktivuj prostředí podle OS
```

### **Bonusové rozšíření**
Vytvoř skript, který automaticky zkontroluje, jestli je virtuální prostředí aktivní a pokud ne, aktivuje ho.

---

## Cvičení 5: Správa balíčků pomocí `requirements.txt`

### **Název úkolu**
Vytvoření a použití souboru `requirements.txt`

### **Detailní zadání**
Student vytvoří virtuální prostředí, nainstaluje několik balíčků (např. `requests`, `numpy`), exportuje seznam do souboru `requirements.txt` a poté odinstaluje balíčky a znovu je nainstaluje z tohoto souboru.

### **Vstupní data/požadavky**
- Aktivní virtuální prostředí
- Balíčky `requests` a `numpy` nainstalované

### **Očekávaný výstup**
- Soubor `requirements.txt`, který obsahuje seznam balíčků a jejich verzí
- Balíčky znovu nainstalované z `requirements.txt`

### **Hints/Nápověda**
1. Nainstaluj balíčky: `pip install requests numpy`
2. Exportuj seznam balíčků: `pip freeze > requirements.txt`
3. Odinstaluj balíčky: `pip uninstall requests numpy`
4. Nainstaluj znovu z `requirements.txt


## 📚 Další zdroje a materiály

<div style="background: #e3f2fd; padding: 20px; border-left: 5px solid #2196f3; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🔗 Doporučené materiály k dalšímu studiu</h3>
</div>

# Zdroje pro kapitolu: **Python – instalace a virtuální prostředí**

---

## 📚 5 doporučených článků / tutoriálů

1. **[Python Installation & Virtual Environments - Real Python](https://realpython.com/python-virtual-environments-a-primer/)**  
   Podrobný úvod do vytváření a správy virtuálních prostředí v Pythonu. Vhodný pro začátečníky i pokročilé.

2. **[Installing Python on Windows, Mac & Linux - Python.org](https://docs.python.org/3/using/index.html)**  
   Oficiální dokumentace od Python.org s podrobnými pokyny k instalaci na různých operačních systémech.

3. **[Python Virtual Environments: A Primer - Corey Schafer](https://coreyms.com/)**  
   Jednoduchý a přehledný článek vysvětlující koncept virtuálních prostředí a jejich výhody.

4. **[Setting Up Python Environment - GeeksforGeeks](https://www.geeksforgeeks.org/setting-up-python-environment/)**  
   Shrnutí základních kroků pro nastavení Python vývojového prostředí, ideální pro začátečníky.

5. **[Python Setup and Virtual Environments - Beginner's Guide to Python](https://github.com/realpython/python-basics)**  
   Příručka od Real Python zaměřená na základy instalace a správy prostředí.

---

## 🎥 3 YouTube videa

1. **[Python Virtual Environments Explained - Corey Schafer (12 min)](https://www.youtube.com/watch?v=Kp4MvWK4evY)**  
   Krátké, ale obsáhlé vysvětlení virtuálních prostředí s praktickým příkladem.

2. **[How to Install Python on Windows 10 / 11 - Programming with Mosh (15 min)](https://www.youtube.com/watch?v=6b43dRQ7F8g)**  
   Zábavný a jednoduchý návod na instalaci Pythonu i nastavení pracovního prostředí.

3. **[Python Virtual Environments - Full Tutorial (20 min)](https://www.youtube.com/watch?v=2D9ZL55F28k)**  
   Detailní video s vysvětlením, jak vytvořit a používat virtuální prostředí pomocí `venv` a `pipenv`.

---

## 📖 3 knihy / oficiální dokumentace

1. **[Automate the Boring Stuff with Python – Al Sweigart](https://automatetheboringstuff.com/)**  
   Kniha pro začátečníky zaměřená na praktické využití Pythonu, včetně základů instalace a práce s prostředím.

2. **[Python 3 Documentation – Official Docs](https://docs.python.org/3/index.html)**  
   Oficiální dokumentace Pythonu, která obsahuje kapitoly o instalaci, `venv` a virtuálních prostředích.

3. **[Effective Python – Brett Slatkin](https://effectivepython.com/)**  
   Kniha zaměřená na lepší způsoby psaní Pythonu. Obsahuje kapitoly o správě závislostí a virtuálních prostředích.

---

## 💻 2 praktické projekty k procvičení

1. **Vytvoření virtuálního prostředí pro projekt "Kalkulačka"**  
   Vytvoř virtuální prostředí, nainstaluj potřebné knihovny (např. `sympy`, `numpy`) a vytvoř jednoduchou kalkulačku s využitím těchto knihoven.

2. **Správa závislostí v projektu**  
   Vytvoř virtuáln


## 📝 Shrnutí kapitoly

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 10px; color: white; margin: 30px 0;">
    <h3 style="color: white; margin-top: 0;">✅ Co jste se naučili</h3>
    <ul style="list-style: none; padding-left: 0;">
        <li>✓ Python 3.11+ instalace</li>
<li>✓ Windows: py launcher, python.exe</li>
<li>✓ MacOS: pyenv, python3</li>
<li>✓ Ubuntu: python3, python3-pip</li>
<li>✓ Virtuální prostředí: venv, virtualenv, conda</li>
    </ul>
    
    <h3 style="color: white;">🎯 Klíčové dovednosti</h3>
    <p>Po dokončení této kapitoly byste měli být schopni prakticky použít všechny probrané koncepty.</p>
    
    <h3 style='color: white;'>➡️ Další kapitola</h3><p>Kapitola 5 - pokračujte ve studiu!</p>
</div>

---

*📅 Notebook vygenerován: 2025-09-29 11:46:23*  
*🤖 Generátor: Comprehensive Colab Generator v2.0*  
*📚 Učebnice programování - Od základů k AI*


## 🧪 Sandbox - Prostor pro experimenty

Použijte následující buňky pro vlastní experimenty a testování:

In [ ]:
# 🧪 Zde můžete experimentovat s kódem z kapitoly
# Napište svůj kód zde:

